<a href="https://colab.research.google.com/github/madelineschwarz/QUAKES-processing-scripts/blob/main/Sample_3DEP_markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

## Sample 3DEP Elevation for Ground Control Points (GCPs)
- QUAKES imagery from initial engineering flights needs manual control points during camera alignment in SFM workflow
- Control points need accurate elevation values to properly model topography
- USGS 3DEP data provides ~1m elevation coverage available thru GEE

Steps:


1.   Upload Control points shapefile containing longitude/latitude positions of control points
2.   Run notebook
3. Download corrected markers from local Collab drive
4. Import into SFM software (e.g. Agisoft Metashape)
5.  Correct update model transform


### Import the API

Run the following cell to import the API into your session.

In [9]:
# To use the Google Collab T4 GPU:
#import tensorflow as tf
# Check GPU availability
#print("GPU Available:", tf.config.experimental.list_physical_devices("GPU"))

In [10]:
import ee
import geopandas
import geemap
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Point
import numpy as np
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='iron-atom-398503')

In [11]:
# function to sample 3DEP data using gdf points
def sample_3dep_data(gdf):
    # Load the 3DEP DEM ImageCollection
    dem_collection = ee.ImageCollection("USGS/3DEP/1m")

    # Convert the GeoDataFrame to an Earth Engine FeatureCollection
    ee_points = geemap.geopandas_to_ee(gdf)
    ee_points = ee.FeatureCollection(ee_points)

    # Convert the image collection into an image by mosaicking (to avoid multiple samples per point)
      #The mosaic() method composites overlapping images according to their order in the collection (last on top).
      # For this use case, we want the most recent lidar survey (1m)
    dem_image = dem_collection.mosaic()

    # Define a function to sample elevation, latitude, and longitude values at the points
    def sample_at_points(feature):
        # Sample elevation at the given point
        sampled_point = dem_image.sample(
            region=feature.geometry(),
            scale=1,  # 1-meter scale for 1-meter DEM data
            geometries=True  # Include the point geometry in the output
        ).first()  # Get the first (and only) sampled point

        # Get elevation value and set it as a property
        elevation = sampled_point.get("elevation")
        return feature.set({"elevation": elevation})

    # Map the sampling function over the FeatureCollection
    sampled_points = ee_points.map(sample_at_points)

    # Convert the result back to a GeoDataFrame
    extracted_gdf = gpd.GeoDataFrame.from_features(sampled_points.getInfo())

    return extracted_gdf

In [12]:
# View the sampled elevation points
# Conversion from gdf to ee object lost the geometry, lat, long columns
markers = gpd.read_file('small_cala_markers.shp')
#print(markers.crs)
#print(markers.head)

new_geom = gpd.points_from_xy(markers.geometry.x, markers.geometry.y, z=None, crs="EPSG:4326" )

new_markers = gpd.GeoDataFrame(geometry = new_geom, crs="EPSG:4326")
#print(new_markers.head
print(type(new_markers))
print(new_markers)
print(new_markers.crs)

<class 'geopandas.geodataframe.GeoDataFrame'>
                      geometry
0  POINT (-121.54399 37.07711)
1  POINT (-121.51009 37.04813)
2   POINT (-121.49128 37.0014)
3  POINT (-121.46934 36.97692)
4  POINT (-121.44633 36.98628)
5  POINT (-121.39954 36.88928)
6  POINT (-121.43487 36.94155)
7  POINT (-121.38565 36.82688)
8  POINT (-121.42655 36.77622)
9  POINT (-121.35852 36.78787)
EPSG:4326


In [13]:
# Convert GeoDataFrame to Earth Engine objects
samples_from_new_markers = sample_3dep_data(new_markers)
print(samples_from_new_markers)



                      geometry   elevation
0  POINT (-121.54399 37.07711)  177.211105
1  POINT (-121.51009 37.04813)  152.014099
2   POINT (-121.49128 37.0014)   70.559555
3  POINT (-121.46934 36.97692)   45.418488
4  POINT (-121.44633 36.98628)   51.099346
5  POINT (-121.39954 36.88928)   69.836609
6  POINT (-121.43487 36.94155)   49.952000
7  POINT (-121.38565 36.82688)  102.577728
8  POINT (-121.42655 36.77622)  296.502930
9  POINT (-121.35852 36.78787)  122.844910


In [14]:
# write new points file that includes newly sampled elevation values
# create point geometry with long, lat, and 3DEP elevation values
elev_markers = gpd.points_from_xy(markers.geometry.x, markers.geometry.y, z=samples_from_new_markers['elevation'], crs="EPSG:4326" )
print(elev_markers)

elev_markers = gpd.GeoDataFrame(geometry = elev_markers, crs="EPSG:4326")
print(elev_markers)
# export elevation markers as a shapefile
elev_markers.to_file('elev_markers.shp')


<GeometryArray>
[<POINT Z (-121.544 37.077 177.211)>,  <POINT Z (-121.51 37.048 152.014)>,
   <POINT Z (-121.491 37.001 70.56)>,  <POINT Z (-121.469 36.977 45.418)>,
  <POINT Z (-121.446 36.986 51.099)>,    <POINT Z (-121.4 36.889 69.837)>,
  <POINT Z (-121.435 36.942 49.952)>, <POINT Z (-121.386 36.827 102.578)>,
 <POINT Z (-121.427 36.776 296.503)>, <POINT Z (-121.359 36.788 122.845)>]
Length: 10, dtype: geometry
                                  geometry
0  POINT Z (-121.54399 37.07711 177.21111)
1   POINT Z (-121.51009 37.04813 152.0141)
2    POINT Z (-121.49128 37.0014 70.55956)
3   POINT Z (-121.46934 36.97692 45.41849)
4   POINT Z (-121.44633 36.98628 51.09935)
5   POINT Z (-121.39954 36.88928 69.83661)
6     POINT Z (-121.43487 36.94155 49.952)
7  POINT Z (-121.38565 36.82688 102.57773)
8  POINT Z (-121.42655 36.77622 296.50293)
9  POINT Z (-121.35852 36.78787 122.84491)


In [ ]:
# convert shp to csv and xml format for Metashape
elev_markers_to_write = gpd.GeoDataFrame()
elev_markers_to_write['Longitude'] = markers.geometry.x
elev_markers_to_write['Latitude'] = markers.geometry.y
elev_markers_to_write['Elevation'] = samples_from_new_markers['elevation']
print(elev_markers_to_write)
elev_markers_to_write.to_csv('elev_markers.csv')